In [10]:
import pandas as pd
import numpy as np

# Load the CSV files
df_final_pt_1 = pd.read_csv('df_final_web_data_pt_1.txt')
df_final_pt_2 = pd.read_csv('df_final_web_data_pt_2.txt')

# Concatenate the two DataFrames into one
df_final_combined = pd.concat([df_final_pt_1, df_final_pt_2], ignore_index=True)

df_final_combined['visit_id'].nunique()

158095

In [9]:
df_final_combined

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
755400,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
755401,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
755402,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
755403,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [4]:
# Convert the 'date_time' column to datetime format
df_final_combined['date_time'] = pd.to_datetime(df_final_combined['date_time'])

# Sort by 'client_id', 'visitor_id', 'visit_id', and 'date_time' in ascending order to ensure durations are positive
df_final_combined = df_final_combined.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'], ascending=[True, True, True, True])

# Calculate the duration between consecutive steps for each visit_id
df_final_combined['step_duration_exact'] = df_final_combined.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff().dt.total_seconds()
df_final_combined['step_duration_exact'].fillna(0, inplace=True)

# Create columns for each step duration
df_final_combined['start_duration'] = df_final_combined.apply(lambda row: row['step_duration_exact'] if row['process_step'] == 'start' else 0, axis=1)
df_final_combined['step1_duration_exact'] = df_final_combined.apply(lambda row: row['step_duration_exact'] if row['process_step'] == 'step_1' else 0, axis=1)
df_final_combined['step2_duration_exact'] = df_final_combined.apply(lambda row: row['step_duration_exact'] if row['process_step'] == 'step_2' else 0, axis=1)
df_final_combined['step3_duration_exact'] = df_final_combined.apply(lambda row: row['step_duration_exact'] if row['process_step'] == 'step_3' else 0, axis=1)
df_final_combined['confirm_duration_exact'] = df_final_combined.apply(lambda row: row['step_duration_exact'] if row['process_step'] == 'confirm' else 0, axis=1)

# Reorder the data by 'date_time' in descending order after calculations
df_final_combined = df_final_combined.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'], ascending=[True, True, True, False])

# Define a function to determine the time period of the day
def time_period(hour):  
    if hour >= 6 and hour <= 9:
        return "early morning"
    elif hour >= 9 and hour <= 12:
        return "mid-morning"
    elif hour >= 12 and hour <= 14:
        return "lunchtime"
    elif hour >= 14 and hour <= 17:
        return "afternoon"
    elif hour >= 17 and hour <= 20:
        return "evening"
    elif hour >= 20 and hour <= 23:
        return "late evening"
    else:
        return "night"

# Extract the hour of day and day period
df_final_combined['hour_of_day'] = df_final_combined['date_time'].dt.hour
df_final_combined['Day period'] = df_final_combined['hour_of_day'].apply(lambda x: time_period(int(x)))

# Correct the Repetition detection logic
df_final_combined['Repetition'] = df_final_combined.groupby('visit_id')['process_step'].transform(lambda x: x.duplicated().any())

# Define a function to count the occurrences of a specific step
def number_of(my_step, column_name, df):
    number_step = df.groupby('visit_id')['process_step'].apply(lambda x: (x == my_step).sum())
    number_step = number_step.reset_index().rename(columns={'process_step': column_name})
    df = df.merge(number_step, on='visit_id', how='left')
    return df

# Count the occurrences of each step
df_final_combined = number_of('start', 'number_start', df_final_combined)
df_final_combined = number_of('step_1', 'number_steps1', df_final_combined)
df_final_combined = number_of('step_2', 'number_steps2', df_final_combined)
df_final_combined = number_of('step_3', 'number_steps3', df_final_combined)
df_final_combined = number_of('confirm', 'number_confirm', df_final_combined)

# Check if 'confirm' is present at least once for each 'visit_id'
groupby_visit_id = df_final_combined.groupby('visit_id')['process_step']
confirm = groupby_visit_id.apply(lambda x: 'confirm' in x.values)

# Add the 'confirm' column to the DataFrame
confirm = confirm.reset_index().rename(columns={'process_step': 'confirm'})
df_final_combined = df_final_combined.merge(confirm, on='visit_id', how='left')

# Group by 'visit_id' and aggregate all necessary columns
df_grouped = df_final_combined.groupby('visit_id').agg({
    'client_id': 'first',
    'visitor_id': 'first',
    'start_duration': 'sum',
    'step1_duration_exact': 'sum',
    'step2_duration_exact': 'sum',
    'step3_duration_exact': 'sum',
    'confirm_duration_exact': 'sum',
    'hour_of_day': 'first',
    'Day period': 'first',
    'Repetition': 'first',
    'confirm': 'first',
    'number_start': 'first',
    'number_steps1': 'first',
    'number_steps2': 'first',
    'number_steps3': 'first',
    'number_confirm': 'first',
})

# Add a new column 'total_steps' as the sum of the 5 step counts
df_grouped['total_steps'] = df_grouped['number_start'] + df_grouped['number_steps1'] + df_grouped['number_steps2'] + df_grouped['number_steps3'] + df_grouped['number_confirm']

# Calculate total duration as the sum of all individual step durations
df_grouped['total_duration'] = df_grouped['start_duration'] + df_grouped['step1_duration_exact'] + df_grouped['step2_duration_exact'] + df_grouped['step3_duration_exact'] + df_grouped['confirm_duration_exact']

# Display the result
df_grouped.head(10)



C:\Users\benoi\AppData\Local\Temp\ipykernel_34328\1204296825.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final_combined['step_duration_exact'].fillna(0, inplace=True)


,client_id,visitor_id,start_duration,step1_duration_exact,step2_duration_exact,step3_duration_exact,confirm_duration_exact,hour_of_day,Day period,Repetition,confirm,number_start,number_steps1,number_steps2,number_steps3,number_confirm,total_steps,total_duration
visit_id,,,,,,,,,,,,,,,,,,
100012776_37918976071_457913,3561384,451664975_1722933822,0.0,0.0,0.0,0.0,52.0,13,lunchtime,True,True,0,0,0,0,2,2,52.0
1000165_4190026492_760066,9056452,306992881_89423906595,0.0,3.0,24.0,77.0,37.0,1,night,False,True,1,1,1,1,1,5,141.0
100019538_17884295066_43909,7338123,612065484_94198474375,18.0,41.0,13.0,113.0,57.0,16,afternoon,True,True,3,4,2,1,1,11,242.0
100022086_87870757897_149620,2478628,754122351_18568832435,0.0,22.0,45.0,53.0,60.0,20,evening,False,True,1,1,1,1,1,5,180.0
100030127_47967100085_936361,105007,676020267_36602664238,0.0,0.0,0.0,0.0,0.0,11,mid-morning,False,False,1,0,0,0,0,1,0.0
100034012_10293842356_627828,5303781,817384618_23455993918,84.0,14.0,5.0,91.0,31.0,14,lunchtime,True,True,2,2,1,1,1,7,225.0
100037962_47432393712_705583,5623007,700426174_33289797318,93.0,39.0,0.0,0.0,0.0,16,afternoon,True,False,3,1,0,0,0,4,132.0
100057941_88477660212_944512,4823947,611052344_43146965242,0.0,57.0,107.0,90.0,0.0,11,mid-morning,True,False,1,2,2,2,0,7,254.0
100061345_677868293_127105,1748647,413751600_31279009983,0.0,0.0,0.0,0.0,0.0,6,early morning,False,False,1,0,0,0,0,1,0.0


In [5]:

# creating transaction status column
#Define the conditions
conditions = [
    (df_grouped['confirm'] == True) & (df_grouped['Repetition'] == False) & (df_grouped['total_steps'] == 5 ),
    (df_grouped['confirm'] == True) & (df_grouped['Repetition'] == True) & (df_grouped['number_start'] != 0) & (df_grouped['number_steps1'] != 0) & (df_grouped['number_steps2'] != 0) & (df_grouped['number_steps3'] != 0)
]
# Define the corresponding values for the new column
choices = [
    'Confirmed w/o Repetition',
    'Confirmed with Repetition',
]
# Create the new column 'transaction_status'
df_grouped['transaction_status'] = np.select(conditions, choices, default='Error')
df_grouped.head()

,client_id,visitor_id,start_duration,step1_duration_exact,step2_duration_exact,step3_duration_exact,confirm_duration_exact,hour_of_day,Day period,Repetition,confirm,number_start,number_steps1,number_steps2,number_steps3,number_confirm,total_steps,total_duration,transaction_status
visit_id,,,,,,,,,,,,,,,,,,,
100012776_37918976071_457913,3561384,451664975_1722933822,0.0,0.0,0.0,0.0,52.0,13,lunchtime,True,True,0,0,0,0,2,2,52.0,Error
1000165_4190026492_760066,9056452,306992881_89423906595,0.0,3.0,24.0,77.0,37.0,1,night,False,True,1,1,1,1,1,5,141.0,Confirmed w/o Repetition
100019538_17884295066_43909,7338123,612065484_94198474375,18.0,41.0,13.0,113.0,57.0,16,afternoon,True,True,3,4,2,1,1,11,242.0,Confirmed with Repetition
100022086_87870757897_149620,2478628,754122351_18568832435,0.0,22.0,45.0,53.0,60.0,20,evening,False,True,1,1,1,1,1,5,180.0,Confirmed w/o Repetition
100030127_47967100085_936361,105007,676020267_36602664238,0.0,0.0,0.0,0.0,0.0,11,mid-morning,False,False,1,0,0,0,0,1,0.0,Error


In [11]:
#check if the resulting df has as many entries as unique visit_id in original df 
df_final_combined['visit_id'].nunique(), len(df_grouped)

(158095, 158095)

In [12]:

# Convert to an excel file
df_grouped.to_excel('web_transactions_summary.xlsx', index=True)